# Klassische Segmentierung
Unter klassischer Segmentierung verstehen wir Methoden der Bildanalyse, die nicht auf den Konzepten von Deep Learning basieren. Hier werden meist Filter und Thresholds verwendet. Hinzu kommen mathematische Operationen mit Bildern und morphologische Operationen.
### Filter
Bei einem Filter wird das Bild in irgendeiner weise weiterverarbeitet. Filter sind meist lokal, d.h. der Wert des neuen Bildes hängt nicht von allen Pixeln des Ausgangsbildes ab, sondern nur von wenigen benachbarten Pixeln. Meist werden Filter daher über Faltungen definiert. Dies sind lineare Filter, bspw. Mean- oder Gauß-Filter. Es gibt jedoch auch Min- oder Median-Filter. Diese sind auch über einen Kernel definiert, können jedoch nicht als Faltung dargestellt werden. Darüberhinaus sind diese nicht-linear.
### Thresholds
Mit Hilfe von Thresholds können Bilder diskretisiert werden. Meist werden sie binärisiert. D.h. aus einem Graustufenbild entsteht ein Bild, in dem jedes Pixel nur noch an oder aus sein kann. Für das Thresholding wird meist ein Wert vorgegeben, alle Pixel, deren Intensität darunter liegt, werden auf 0, alle anderen auf 1 gesetzt. Es gibt verschiedene Methoden, um diesen Wert automatisiert zu bestimmen. Meist basieren diese Methoden auf dem Histogram eines Bildes.
### Mathematische Operationen
Oftmals werden Bilder gefiltert und dann mit einander addiert oder multipliziert. So kann bspw. Rauschen oder das Hintergrundsignal eliminiert werden. Bei diesen Operationen sollte unbedingt auf den Datentyp der Bilder geachtet werden!!!
### morphologische Operationen
Auf Binärbildern kann man auch Min- und Max-Filter anwenden. Diese Operationen heißen hier dann Erosion und Dilatation. Durch Hintereinanderausführen kann man daraus auch die morphologischen Operationen Opening und Closing definieren.

## Workflow
Der klassische Workflow sieht meist vor, dass ein Bild mit Hilfe eines Filters entrauscht wird. Danach werden durch einen weiteren Filter und mathematische Operationen die Kanten des Bildes hervorgehoben. Anschließend wird das Bild mittels Thresholding binärisiert. Schlussendlich wird das Binärbild nochmals bearbeitet.

In [ ]:
import scipy
import skimage
import skimage.filters
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tqdm import tqdm
from glob import glob
from tifffile import imread
from skimage.morphology import disk

from stardist import random_label_cmap
from stardist.matching import matching_dataset
from csbdeep.utils import Path
lbl_cmap = random_label_cmap()

In [ ]:
# Damit wir den Effekt von Filtern nachvollziehen können schreiben wir eine kleine Funktion,
# um mehrere Bilder mit verschiedenen Titeln nebeneinander darstellen zu können.
def plot_effect(imgs, labels, cmap=None):
    cols = len(imgs)
    rows = len(imgs[0])
    fig, axes = plt.subplots(cols, rows, figsize=[16, cols*5])
    for i in range(cols):
        for j in range(rows):
            try:
                ax = axes[i, j]
            except IndexError:
                ax = axes[j]
            ax.imshow(imgs[i][j], cmap=cmap)
            ax.axis("off")
            ax.set_title(labels[i][j])

Wir laden wieder unsere Trainings-Daten und benutzen für den weiteren Verlauf einen Ausschnitt des ersten Bilds.

In [ ]:
X_glob = sorted(glob('/extdata/readonly/f-prak-v15/e-coli-swarming/train/input/*.tif'))
Y_glob = sorted(glob('/extdata/readonly/f-prak-v15/e-coli-swarming/train/labels/*.tif'))
X = [x.astype(int) for x in list(map(imread, X_glob))]
Y = [x.astype(int) for x in list(map(imread, Y_glob))]
sly = slice(380, 508)
slx = slice(380, 508)
img, lbl = X[0][sly, slx], Y[0][sly, slx]

In [ ]:
skimage.__version__

In [ ]:
median = skimage.filters.median(img, disk(5))
mean = scipy.ndimage.filters.uniform_filter(img, 5)
gauss = skimage.filters.gaussian(img, 2)
diff_gauss = skimage.filters.difference_of_gaussians(img, 1, 20)
black_tophat = scipy.ndimage.morphology.black_tophat(img, 5)
white_tophat = scipy.ndimage.morphology.white_tophat(img, 5)
plot_effect(
    [
        [img, median, img-median],
        [img, mean, img-mean],
        [img, gauss, img-gauss],
        [img, diff_gauss, img-diff_gauss],
        [img, black_tophat, img-black_tophat],
        [img, white_tophat, img-white_tophat],
    ],
    [
        ["input", "median filter", "input - median filter"],
        ["input", "mean filter", "input - mean filter"],
        ["input", "gauss filter", "input - gauss filter"],
        ["input", "diff gauss filter", "input - diff gauss filter"],
        ["input", "black_tophat filter", "input - black_tophat filter"],
        ["input", "white_tophat filter", "input - white_tophat filter"],
    ]
)

In [ ]:
plt.hist(img.flatten(), bins=50)
None;

In [ ]:
imgs = []
labels = []
low_sigmas = [1, 2, 3, 4, 5, 10]
high_sigma = 20
for low_sigma in low_sigmas:
    diff_gauss = skimage.filters.difference_of_gaussians(img, low_sigma, high_sigma)
    imgs.append([
        img, diff_gauss, img-diff_gauss
    ])
    labels.append([
        "input",
        "low_sigma={}, high_sigma={}".format(low_sigma, high_sigma),
        "input - diff_gauss"
    ])
plot_effect(imgs, labels)

In [ ]:
imgs = []
labels = []
sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for size in sizes:
    white_tophat = scipy.ndimage.morphology.white_tophat(img, size)
    median = skimage.filters.median(white_tophat, disk(5))
    imgs.append([
        img, white_tophat, median
    ])
    labels.append([
        "input",
        "size = {}".format(size),
        "median of tophat"
    ])
plot_effect(imgs, labels)

In [ ]:
tophat = scipy.ndimage.morphology.white_tophat(tmp_img, 10)
tophat_median = skimage.filters.median(tophat, disk(5))
plt.figure(figsize=[16, 10])
plt.subplot(211)
plt.hist(img.flatten(), bins=50)
plt.subplot(212)
plt.hist(np.exp(tophat.flatten()*1e16), bins=50)
None;

In [ ]:
tmp_img = skimage.filters.gaussian(img, 2)
imgs = []
labels = []
sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for size in sizes:
    white_tophat = scipy.ndimage.morphology.white_tophat(tmp_img, 10)
    black_tophat = scipy.ndimage.morphology.black_tophat(tmp_img, size)
    #black_tophat = skimage.filters.median(white_tophat, disk(5))
    imgs.append([
        tmp_img, white_tophat, black_tophat, black_tophat+white_tophat, white_tophat-black_tophat
    ])
    labels.append([
        "size = {}".format(size),
        "white tophat", "black tophat", "sum", "diff"
    ])
plot_effect(imgs, labels)

In [ ]:
gauss = skimage.filters.gaussian(img, 2)
white_tophat = scipy.ndimage.morphology.white_tophat(gauss, 10)
thresh = skimage.filters.threshold_otsu(white_tophat)
binary = white_tophat > thresh
plot_effect([[img, gauss, white_tophat, binary]], [["input", "gauss", "white_tophat", "otsu"]])

In [ ]:
tmp_img = X[0][200:800, 200:800]
gt = Y[0][200:800, 200:800]
gauss = skimage.filters.gaussian(tmp_img, 2)
white_tophat = scipy.ndimage.morphology.white_tophat(gauss, 10)
thresh = skimage.filters.threshold_otsu(white_tophat)
binary = white_tophat > thresh
labelmap = skimage.measure.label(binary)
plot_effect([[tmp_img, labelmap, gt]], [["input", "seg", "label"]])

In [ ]:
region_gt = skimage.measure.regionprops(gt)
region_seg = skimage.measure.regionprops(labelmap)
print(len(region_gt))
print(len(region_seg))

In [ ]:
taus = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
stats = [matching_dataset(gt, labelmap, thresh=t, show_progress=False) for t in tqdm(taus)]

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(15,5))

for m in ('precision', 'recall', 'accuracy', 'f1', 'mean_true_score'):
    ax1.plot(taus, [s._asdict()[m] for s in stats], '.-', lw=2, label=m)
ax1.set_xlabel(r'IoU threshold $\tau$')
ax1.set_ylabel('Metric value')
ax1.grid()
ax1.legend()

for m in ('fp', 'tp', 'fn'):
    ax2.plot(taus, [s._asdict()[m] for s in stats], '.-', lw=2, label=m)
ax2.set_xlabel(r'IoU threshold $\tau$')
ax2.set_ylabel('Number #')
ax2.grid()
ax2.legend();